## Task 1: Parsing table from wikipedia

In [4]:
#setup
import bs4
from bs4 import BeautifulSoup as soup
import requests
import lxml.html as lh
import pandas as pd
from urllib.request import urlopen, Request
import numpy as np
import json
!conda install --channel conda-forge geopandas geoplot geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [10]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [11]:
tr_elements = doc.xpath('//tr')
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal code
"
2:"Borough
"
3:"Neighborhood
"


In [12]:
#Get table rows
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break    
    i=0   
    for t in T.iterchildren():
        data=t.text_content() 
        col[i][1].append(data)
        i+=1
Dict={title:column for (title,column) in col}

#To Dataframe
df=pd.DataFrame(Dict)
df.replace(r'\s', '', regex = True, inplace = True)
df.columns = df.columns.str.rstrip()
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Notassigned,
1,M2A,Notassigned,
2,M3A,NorthYork,Parkwoods
3,M4A,NorthYork,VictoriaVillage
4,M5A,DowntownToronto,RegentPark/Harbourfront


In [38]:
#Remove Notassigned rows
new_df = df[df['Borough'] != 'Notassigned']
new_df.columns = ['PostalCode', 'Borough','Neighborhood']
new_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,NorthYork,Parkwoods
3,M4A,NorthYork,VictoriaVillage
4,M5A,DowntownToronto,RegentPark/Harbourfront
5,M6A,NorthYork,LawrenceManor/LawrenceHeights
6,M7A,DowntownToronto,Queen'sPark/OntarioProvincialGovernment


In [34]:
new_df.shape

(104, 3)

## Task 2 Get geographical coordinates of each postal code

In [39]:
#Convert csv to data frame

geospatial_df = pd.read_csv(r'http://cocl.us/Geospatial_data')
geospatial_df.columns = ['PostalCode', 'Latitude','Longtitude']
geospatial_df.head()

,PostalCode,Latitude,Longtitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
# Merge geospatial data with current dataframe

df3 = pd.merge(new_df,geospatial_df)
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longtitude
0,M3A,NorthYork,Parkwoods,43.753259,-79.329656
1,M4A,NorthYork,VictoriaVillage,43.725882,-79.315572
2,M5A,DowntownToronto,RegentPark/Harbourfront,43.654260,-79.360636
3,M6A,NorthYork,LawrenceManor/LawrenceHeights,43.718518,-79.464763
4,M7A,DowntownToronto,Queen'sPark/OntarioProvincialGovernment,43.662301,-79.389494


In [64]:
#Filter to work with Toronto only

df4 = df3[df3['Borough'].str.contains("Toronto")]
df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longtitude
2,M5A,DowntownToronto,RegentPark/Harbourfront,43.654260,-79.360636
4,M7A,DowntownToronto,Queen'sPark/OntarioProvincialGovernment,43.662301,-79.389494
9,M5B,DowntownToronto,"GardenDistrict,Ryerson",43.657162,-79.378937
15,M5C,DowntownToronto,St.JamesTown,43.651494,-79.375418
19,M4E,EastToronto,TheBeaches,43.676357,-79.293031


## Task 3 Analysis of data

In [57]:
#Get geographical cooordinates of Toronto
address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.8184944, -79.3309468.


In [114]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longtitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [95]:
#Create a new dataframe of the Downtown Toronto data.
downtowntoronto_data = df4[df4['Borough'] == 'DowntownToronto'].reset_index(drop=True)
downtowntoronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longtitude
0,M5A,DowntownToronto,RegentPark/Harbourfront,43.654260,-79.360636
1,M7A,DowntownToronto,Queen'sPark/OntarioProvincialGovernment,43.662301,-79.389494
2,M5B,DowntownToronto,"GardenDistrict,Ryerson",43.657162,-79.378937
3,M5C,DowntownToronto,St.JamesTown,43.651494,-79.375418
4,M5E,DowntownToronto,BerczyPark,43.644771,-79.373306


In [107]:
latitude = 43.6515
longitude = -79.3835
print('The geograpical coordinate of DowntownToroto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of DowntownToroto are 43.6515, -79.3835.


In [121]:
# create map of Downtown Toronto using latitude and longitude values
map_downtowntoronto = folium.Map(location=[latitude, longitude], zoom_start=13)
for lat, lng, label in zip(downtowntoronto_data['Latitude'], downtowntoronto_data['Longtitude'], downtowntoronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtowntoronto)  
    
map_downtowntoronto

In [111]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'PQQCNR0HQMW1AD322IIDQ5I4PD3EA4XBV0NNWOEWZEXILBKQ' 
CLIENT_SECRET = 'IV4B0HMBVYPIWT4PQ2TREBOMG5K2UAJANN1BKXCJHGBRBPOJ'
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PQQCNR0HQMW1AD322IIDQ5I4PD3EA4XBV0NNWOEWZEXILBKQ
CLIENT_SECRET:IV4B0HMBVYPIWT4PQ2TREBOMG5K2UAJANN1BKXCJHGBRBPOJ


In [122]:
downtowntoronto_data.loc[0, 'Neighborhood']

'RegentPark/Harbourfront'

In [124]:
#Explore neighborhood in Downtown Toronto
#Create function
LIMIT = 2

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [125]:
downtowntoronto_venues = getNearbyVenues(names=downtowntoronto_data['Neighborhood'],
                                   latitudes=downtowntoronto_data['Latitude'],
                                   longitudes=downtowntoronto_data['Longtitude']
                                  )

RegentPark/Harbourfront
Queen'sPark/OntarioProvincialGovernment
GardenDistrict,Ryerson
St.JamesTown
BerczyPark
CentralBayStreet
Christie
Richmond/Adelaide/King
HarbourfrontEast/UnionStation/TorontoIslands
TorontoDominionCentre/DesignExchange
CommerceCourt/VictoriaHotel
UniversityofToronto/Harbord
KensingtonMarket/Chinatown/GrangePark
CNTower/KingandSpadina/RailwayLands/HarbourfrontWest/BathurstQuay/SouthNiagara/Islandairport
Rosedale
StnAPOBoxes
St.JamesTown/Cabbagetown
FirstCanadianPlace/Undergroundcity
ChurchandWellesley


In [126]:
print(downtowntoronto_venues.shape)
downtowntoronto_venues.head()

(38, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,RegentPark/Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,RegentPark/Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Queen'sPark/OntarioProvincialGovernment,43.662301,-79.389494,Queen's Park,43.663946,-79.392180,Park
3,Queen'sPark/OntarioProvincialGovernment,43.662301,-79.389494,Sushi Box,43.662960,-79.386580,Sushi Restaurant
4,"GardenDistrict,Ryerson",43.657162,-79.378937,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store


In [127]:
downtowntoronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BerczyPark,2,2,2,2,2,2
CNTower/KingandSpadina/RailwayLands/HarbourfrontWest/BathurstQuay/SouthNiagara/Islandairport,2,2,2,2,2,2
CentralBayStreet,2,2,2,2,2,2
Christie,2,2,2,2,2,2
ChurchandWellesley,2,2,2,2,2,2
CommerceCourt/VictoriaHotel,2,2,2,2,2,2
FirstCanadianPlace/Undergroundcity,2,2,2,2,2,2
"GardenDistrict,Ryerson",2,2,2,2,2,2
HarbourfrontEast/UnionStation/TorontoIslands,2,2,2,2,2,2


In [130]:
# one hot encoding to analyse each neighborhood
toronto_onehot = pd.get_dummies(downtowntoronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = downtowntoronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Vegetarian / Vegan Restaurant,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Café,Clothing Store,Coffee Shop,Concert Hall,Dance Studio,...,Museum,Neighborhood,Organic Grocery,Park,Playground,Plaza,Restaurant,Sushi Restaurant,Tea Room,Theme Restaurant
0,0,0,0,0,1,0,0,0,0,0,...,0,RegentPark/Harbourfront,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,RegentPark/Harbourfront,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,Queen'sPark/OntarioProvincialGovernment,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,Queen'sPark/OntarioProvincialGovernment,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,"GardenDistrict,Ryerson",0,0,0,0,0,0,0,0


In [133]:
toronto_onehot.shape

(38, 27)

In [132]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
toronto_grouped.shape

(19, 27)

In [149]:
#Create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,BerczyPark,Restaurant,Liquor Store,Theme Restaurant,Grocery Store,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Café,Clothing Store
1,CNTower/KingandSpadina/RailwayLands/Harbourfro...,Airport,Airport Lounge,Theme Restaurant,Tea Room,Arts & Crafts Store,Bakery,Café,Clothing Store,Coffee Shop,Concert Hall
2,CentralBayStreet,Coffee Shop,Theme Restaurant,Tea Room,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Café,Clothing Store,Concert Hall
3,Christie,Café,Grocery Store,Theme Restaurant,Tea Room,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Clothing Store,Coffee Shop
4,ChurchandWellesley,Theme Restaurant,Dance Studio,Tea Room,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Café,Clothing Store,Coffee Shop


In [144]:
#Clustering neighborhoods

kclusters = 5
downtowntoronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtowntoronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 0, 2, 4, 3, 1, 1, 0, 0, 0], dtype=int32)

In [148]:
# Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtowntoronto_merged = downtowntoronto_data

downtowntoronto_merged = downtowntoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtowntoronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longtitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,DowntownToronto,RegentPark/Harbourfront,43.654260,-79.360636,2,Bakery,Coffee Shop,Theme Restaurant,Tea Room,Airport,Airport Lounge,Arts & Crafts Store,Café,Clothing Store,Concert Hall
1,M7A,DowntownToronto,Queen'sPark/OntarioProvincialGovernment,43.662301,-79.389494,0,Sushi Restaurant,Park,Theme Restaurant,Grocery Store,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Café,Clothing Store
2,M5B,DowntownToronto,"GardenDistrict,Ryerson",43.657162,-79.378937,0,Tea Room,Clothing Store,Theme Restaurant,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Café,Coffee Shop,Concert Hall
3,M5C,DowntownToronto,St.JamesTown,43.651494,-79.375418,2,Coffee Shop,Japanese Restaurant,Theme Restaurant,Grocery Store,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Café,Clothing Store
4,M5E,DowntownToronto,BerczyPark,43.644771,-79.373306,1,Restaurant,Liquor Store,Theme Restaurant,Grocery Store,Airport,Airport Lounge,Arts & Crafts Store,Bakery,Café,Clothing Store


In [147]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(downtowntoronto_merged['Latitude'], downtowntoronto_merged['Longtitude'], downtowntoronto_merged['Neighborhood'], downtowntoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters